In [ ]:
#Importation des differentes packages
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets

In [1]:
#Importation des données de metagenomique en un dataframe pandas
df = pd.read_csv("1335.vdb.tab", sep ='\t')

#decoupage des noms des bacteries

l1= []
l2= []
l3= []
l4= []
l5= []
l6= []

for nom in df['OTUConTaxonomy'].tolist():
    l1.append( nom.split(";")[0].split("(")[0])
    l2.append( nom.split(";")[1].split("(")[0])
    l3.append( nom.split(";")[2].split("(")[0])
    l4.append( nom.split(";")[3].split("(")[0])
    l5.append( nom.split(";")[4].split("(")[0])
    l6.append( nom.split(";")[5].split("(")[0])

df['regne'] = l1
df['embranchement']= l2
df['classe'] = l3
df['ordre']= l4
df['famille']= l5
df['genre']= l6

df.head()

,OTUNumber,OTUConTaxonomy,door_in_1,door_in_2,faucet_handle_1,faucet_handle_2,sink_floor_1,sink_floor_2,soap_dispenser_1,stall_in_1,...,toilet_flush_handle_1,toilet_flush_handle_2,toilet_seat_1,toilet_seat_2,regne,embranchement,classe,ordre,famille,genre
0,Otu00001,Bacteria(100);Actinobacteria(100);Actinobacter...,1381,1032,335,577,38,36,1115,598,...,13,329,76,156,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Propionibacteriaceae,Propionibacterium
1,Otu00002,Bacteria(100);Proteobacteria(100);Gammaproteob...,20,17,1686,3,411,564,128,9,...,0,25,258,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Enhydrobacter
2,Otu00003,Bacteria(100);Actinobacteria(100);Actinobacter...,8,47,6,418,15,8,84,45,...,3,283,41,1059,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium
3,Otu00004,Bacteria(100);Actinobacteria(100);Actinobacter...,6,27,5,6,452,275,63,0,...,9,51,5,3,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Micrococcaceae,Arthrobacter
4,Otu00005,Bacteria(100);Actinobacteria(100);Actinobacter...,0,29,0,648,2,8,3,32,...,0,206,7,95,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium


In [13]:
#heatmap

#df_short = df[['door_in_1','door_in_2','faucet_handle_1','faucet_handle_2','sink_floor_1','sink_floor_2','soap_dispenser_1','stall_in_1','toilet_floor_1','toilet_floor_2','toilet_flush_handle_1','toilet_flush_handle_2','toilet_seat_1','toilet_seat_2']]

#remove line with 0
#df_short_clean = df_short.loc[(df.sum(axis=1) != 0)]
#matrice=df.groupby(by="embranchement").sum()


indexNamesArr = matrice.index.values

embranchement = list(indexNamesArr)
samples = list(matrice.columns.values)


fig = go.Figure(data=go.Heatmap(
                   z=matrice,
                   x=samples,
                   y=embranchement,
                   hoverongaps = False))
fig.show()

In [14]:
#Objectif : Visualisation de la repartition des differents embranchements au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="embranchement").sum()


indexNamesArr = matrice.index.values

embranchement = list(indexNamesArr)
samples = list(matrice.columns.values)



fig = px.bar(matrice.transpose(), 
             x=samples, 
             y=embranchement, 
             color_discrete_sequence=px.colors.sequential.Viridis, 
             title="Répartition des differents embranchements")
fig.show()

In [15]:
#Objectif : Visualisation de la repartition des differentes souches au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="embranchement").sum()


indexNamesArr = matrice.index.values

embranchement = list(indexNamesArr)
samples = list(matrice.columns.values)


fig = px.bar(matrice, 
            x=embranchement,
            y=samples, 
             
             color_discrete_sequence=px.colors.sequential.Viridis, 
             title="Répartition des differents embranchements")
fig.show()

In [5]:
#Objectif : Visualisation de la variance des differents embranchements

matrice=df.groupby(by="embranchement").sum()
fig = px.box(matrice, 
             x=embranchement,
             y=sample,
             color=embranchement,
             title="Variance des differents regnes")
fig.update_layout(showlegend=False)
fig.show()

In [7]:
#Pie chart avec des differentes embranchement et bacteries

df['name']= df['famille']+' '+df['genre'] 

sample = 'door_in_1'

fig = px.sunburst(df, 
                  path=['embranchement', 'name'], 
                  values=sample,
                  color_discrete_sequence=px.colors.sequential.Viridis, 
                  title="Repartition des bacteries dans l'echantillon "+sample,
                  ) 
fig.show()

In [16]:
#test subplot avec un pie chart
from plotly.subplots import make_subplots

df['name']= df['famille']+' '+df['genre'] 
matrice=df.groupby(by="embranchement").sum()

embranchement = list(indexNamesArr)
samples = list(matrice.columns.values)


fig = make_subplots(rows=1, cols=len(samples))
c = 1
for sample in samples:
    fig.add_trace(
        px.sunburst(df, 
                      path=['embranchement', 'name'], 
                      values=sample,
                      color_discrete_sequence=px.colors.sequential.Viridis, 
                      title="Repartition des bacteries dans l'echantillon "+sample,
                      ),
        row=1, 
        col=c)
    print(c)
    c+=1

fig.update_layout()
fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'branchvalues': 'total',
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('labels=%{label}<br>door_in_1=%' ... 'nt}<br>id=%{id}<extra></extra>'),
              'ids': array(['Proteobacteria/Acetobacteraceae Acetobacter',
                            'Proteobacteria/Acetobacteraceae Belnapia',
                            'Proteobacteria/Acetobacteraceae Endobacter', ..., 'Synergistetes',
                            'Verrucomicrobia', 'unclassified'], dtype=object),
              'labels': array(['Acetobacteraceae Acetobacter', 'Acetobacteraceae Belnapia',
                               'Acetobacteraceae Endobacter', ..., 'Synergistetes', 'Verrucomicrobia',
                               'unclassified'], dtype=object),
              'name': '',
              'parents': array(['Proteobacteria', 'Proteobacteria', 'Prote

In [17]:
#Test de l'interactivité avec ipywidget

#https://plotly.com/python/figurewidget-app/

df['name']= df['famille']+' '+df['genre'] 
matrice=df.groupby(by="embranchement").sum()

embranchement = list(indexNamesArr)
matrice.to_numpy
columnsNamesArr = matrice.columns.values
samples = list(columnsNamesArr)


df['name']= df['famille']+' '+df['genre']

choice = widgets.Dropdown(
    options=list(samples),
    value='door_in_1',
    description='Echantillon : ',
)


# Assign an empty figure widget with two traces
g = go.FigureWidget(px.sunburst(df, 
                  path=['embranchement', 'name'], 
                  values=choice.value,
                  color_discrete_sequence=px.colors.sequential.Viridis, 
                  title="Repartition des bacteries dans l'echantillon "+sample,
                  ) 
                   )
widgets.VBox([choice])

g

FigureWidget({
    'data': [{'branchvalues': 'total',
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0…